not classify star and circles is unkonw 

In [2]:
import cv2
import numpy as np
import os

# Directory containing the images
image_dir = r"D:\Vortex\ROV\Phase 2\computer vision\Tasks2025\task 3"

# Function to detect and classify shapes based on contour
def classify_shape(contour):
    perimeter = cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, 0.04 * perimeter, True)
    
    vertices = len(approx)
    if vertices == 3:
        return "Triangle"
    elif vertices == 4:
        (x, y, w, h) = cv2.boundingRect(approx)
        aspect_ratio = float(w) / h
        return "Square" if 0.95 <= aspect_ratio <= 1.05 else "Rectangle"
    elif vertices == 10:
        return "Star"
    elif vertices > 10:
        return "Circle"
    else:
        return "Unknown"

# Function to perform shape detection on each image
def process_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Could not read {image_path}")
        return

    # Convert to HSV for color segmentation
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Preprocessing: Gaussian Blur
    blurred = cv2.GaussianBlur(hsv, (5, 5), 0)

    # Define color ranges (adjust if necessary for specific colors)
    color_ranges = [
        ((0, 100, 100), (10, 255, 255)),    # Red
        ((100, 100, 100), (130, 255, 255)),  # Blue
        ((20, 100, 100), (30, 255, 255)),    # Yellow (adjust for orange star)
        ((40, 100, 100), (70, 255, 255)),    # Green
    ]
    
    # Iterate over each color range to detect shapes
    for (lower, upper) in color_ranges:
        mask = cv2.inRange(blurred, lower, upper)
        
        # Erosion and dilation to remove noise
        mask = cv2.erode(mask, None, iterations=2)
        mask = cv2.dilate(mask, None, iterations=2)

        # Find contours in the mask
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        for contour in contours:
            if cv2.contourArea(contour) < 500:  # Ignore small contours
                continue
            
            shape = classify_shape(contour)
            
            # Get the contour's centroid for labeling
            M = cv2.moments(contour)
            if M["m00"] != 0:
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
            else:
                cX, cY = 0, 0

            # Draw the contour and label the shape
            cv2.drawContours(image, [contour], -1, (0, 255, 0), 2)
            cv2.putText(image, shape, (cX - 20, cY - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Show the image with detected shapes
    cv2.imshow("Shape Detection", image)
    cv2.waitKey(0)

# Process each image in the directory
for image_name in os.listdir(image_dir):
    image_path = os.path.join(image_dir, image_name)
    process_image(image_path)

cv2.destroyAllWindows()


Could not read D:\Vortex\ROV\Phase 2\computer vision\Tasks2025\task 3\new


detect stars but has errors

In [3]:
import cv2
import numpy as np
import os

# Directory containing the images
image_dir = r"D:\Vortex\ROV\Phase 2\computer vision\Tasks2025\task 3"

# Function to detect and classify shapes based on contour
def classify_shape(contour):
    perimeter = cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, 0.04 * perimeter, True)
    area = cv2.contourArea(contour)
    vertices = len(approx)
    
    # Calculate roundness for circle detection
    if perimeter == 0:
        return "Unknown"
    roundness = 4 * np.pi * area / (perimeter ** 2)
    
    if vertices == 3:
        return "Triangle"
    elif vertices == 4:
        (x, y, w, h) = cv2.boundingRect(approx)
        aspect_ratio = float(w) / h
        return "Square" if 0.95 <= aspect_ratio <= 1.05 else "Rectangle"
    elif 8 <= vertices <= 12 and roundness > 0.7:
        return "Circle"
    elif 9 <= vertices <= 12:
        return "Star"
    else:
        return "Unknown"

# Function to perform shape detection on each image
def process_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Could not read {image_path}")
        return

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Use adaptive thresholding for edge detection
    adaptive_thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                            cv2.THRESH_BINARY_INV, 11, 2)

    # Find edges using Canny edge detector
    edges = cv2.Canny(adaptive_thresh, 100, 200)

    # Dilate the edges to connect contours
    dilated_edges = cv2.dilate(edges, None, iterations=1)

    # Find contours in the dilated edges
    contours, _ = cv2.findContours(dilated_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        if cv2.contourArea(contour) < 500:  # Ignore small contours
            continue
        
        shape = classify_shape(contour)

        # Get the contour's centroid for labeling
        M = cv2.moments(contour)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
        else:
            cX, cY = 0, 0

        # Draw the contour and label the shape
        cv2.drawContours(image, [contour], -1, (0, 255, 0), 2)
        cv2.putText(image, shape, (cX - 20, cY - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Show the image with detected shapes
    cv2.imshow("Shape Detection", image)
    cv2.waitKey(0)

# Process each image in the directory
for image_name in os.listdir(image_dir):
    image_path = os.path.join(image_dir, image_name)
    process_image(image_path)

cv2.destroyAllWindows()


Could not read D:\Vortex\ROV\Phase 2\computer vision\Tasks2025\task 3\new


detect circles and shapes but stars not classify it

In [4]:
import cv2
import numpy as np
import os

# Directory containing the images
image_dir = r"D:\Vortex\ROV\Phase 2\computer vision\Tasks2025\task 3"

# Function to detect and classify shapes based on contour
def classify_shape(contour):
    perimeter = cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, 0.04 * perimeter, True)
    
    vertices = len(approx)
    if vertices == 3:
        return "Triangle"
    elif vertices == 4:
        (x, y, w, h) = cv2.boundingRect(approx)
        aspect_ratio = float(w) / h
        return "Square" if 0.95 <= aspect_ratio <= 1.05 else "Rectangle"
    elif vertices == 5:
        return "Pentagon"
    elif vertices > 5:
        # Check for circle: calculate the circularity
        area = cv2.contourArea(contour)
        if area == 0:
            return "Unknown"
        circularity = (4 * np.pi * area) / (perimeter ** 2)
        if circularity > 0.8:  # Threshold for circularity (adjust if needed)
            return "Circle"
        else:
            return "Unknown"
    elif vertices == 10:
        return "Star"
    else:
        return "Unknown"

# Function to perform shape detection on each image
def process_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Could not read {image_path}")
        return

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Use adaptive thresholding for edge detection
    adaptive_thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                            cv2.THRESH_BINARY_INV, 11, 2)

    # Find edges using Canny edge detector
    edges = cv2.Canny(adaptive_thresh, 100, 200)

    # Dilate the edges to connect contours
    dilated_edges = cv2.dilate(edges, None, iterations=1)

    # Find contours in the dilated edges
    contours, _ = cv2.findContours(dilated_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        if cv2.contourArea(contour) < 500:  # Ignore small contours
            continue
        
        shape = classify_shape(contour)

        # Get the contour's centroid for labeling
        M = cv2.moments(contour)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
        else:
            cX, cY = 0, 0

        # Draw the contour and label the shape
        cv2.drawContours(image, [contour], -1, (0, 255, 0), 2)
        cv2.putText(image, shape, (cX - 20, cY - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Show the image with detected shapes
    cv2.imshow("Shape Detection", image)
    cv2.waitKey(0)

# Process each image in the directory
for image_name in os.listdir(image_dir):
    image_path = os.path.join(image_dir, image_name)
    process_image(image_path)

cv2.destroyAllWindows()


Could not read D:\Vortex\ROV\Phase 2\computer vision\Tasks2025\task 3\new
